In [1]:
!pip install PyMuPDF spacy
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 58.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [14]:
import zipfile
import os

zip_path = '/content/aresumes.zip'
extract_to = '/content/aresumes_folder'

# Create target directory if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Unzipped to: {extract_to}")


Unzipped to: /content/aresumes_folder


In [15]:
import fitz  # PyMuPDF
import re
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def extract_email(text):
    match = re.search(r'[\w\.-]+@[\w\.-]+', text)
    return match.group() if match else ""

def extract_phone(text):
    match = re.search(r'\+?\d[\d\s\-]{8,15}', text)
    return match.group() if match else ""

def extract_links(text):
    links = []

    # Match LinkedIn
    linkedin_match = re.search(r'(linkedin\.com/in/[a-zA-Z0-9_-]+)', text, re.IGNORECASE)
    if linkedin_match:
        links.append("https://" + linkedin_match.group(1))

    # Match GitHub
    github_match = re.search(r'(github\.com/[a-zA-Z0-9_-]+)', text, re.IGNORECASE)
    if github_match:
        links.append("https://" + github_match.group(1))

    return links

def extract_name(text):
    lines = text.strip().split('\n')[:10]
    for line in lines:
        clean = line.strip()
        # Ignore if line contains contact info or known words
        if "@" in clean or any(x in clean.lower() for x in ["linkedin", "github", "email", "phone", "curriculum", "vitae"]):
            continue
        words = clean.split()
        if 1 < len(words) <= 4 and all(w[0].isupper() for w in words if w):
            return clean
    return ""


def extract_skills(text):
    skill_keywords = [
    "Python", "SQL", "Tableau", "Power BI", "R", "Pandas", "NumPy", "Scikit-learn",
    "MySQL", "PostgreSQL", "Excel", "Java", "C++", "Machine Learning", "Deep Learning",
    "Statistics", "Data Visualization", "Data Mining", "Big Data", "Hadoop", "Spark",
    "Airflow", "Git", "Docker", "Kubernetes", "Data Wrangling", "Data Cleaning",
    "Natural Language Processing", "Computer Vision", "Seaborn", "Matplotlib", "Plotly",
    "ETL Pipelines", "AWS", "Azure", "Google Cloud Platform (GCP)", "Feature Engineering",
    "Model Evaluation", "Model Deployment", "MLOps", "Data Lakes", "Snowflake",
    "Data Governance", "A/B Testing", "Data Ethics", "Bayesian Inference", "EDA",
    "Transformers", "Hugging Face", "FastAPI", "Dask"
    ]
    skill_keywords = [skill.lower() for skill in skill_keywords]
    text = text.lower()
    return [skill for skill in skill_keywords if skill in text]

import re

def extract_degrees(raw_text):
    """
    Extracts academic degree information from raw CV/resume text.

    Args:
        raw_text (str): The raw text from a CV or resume.

    Returns:
        List[str]: A list of extracted degree strings.
    """
    degree_pattern = re.compile(
        r'\b(Bachelor|Master|Doctor|Associate)[\w\s]* in [^\n]+',
        re.IGNORECASE
    )
    match = degree_pattern.search(raw_text)
    return match.group().strip() if match else ''






def extract_experience_years(text):
    match = re.findall(r'(\d+)\+?\s*years?', text.lower())
    return max([int(m) for m in match], default=0)

def extract_role(text):
    role_keywords = [
    "Data Analyst", "Data Scientist", "Business Analyst", "Machine Learning Engineer",
    "AI Engineer", "BI Developer", "Quantitative Analyst", "Data Engineer",
    "Big Data Engineer", "Research Scientist", "Analytics Consultant",
    "Decision Scientist", "Data Architect", "Data Strategist", "Statistical Programmer",
    "ML Ops Engineer", "NLP Engineer", "Computer Vision Engineer", "Product Data Analyst",
    "Data Journalist", "Business Intelligence Analyst", "Healthcare Data Analyst",
    "Customer Insights Analyst", "Operations Analyst", "Marketing Data Analyst",
    "Financial Data Analyst", "Growth Analyst", "Risk Analyst", "Fraud Analyst",
    "Research Analyst", "CRM Analyst", "Data Quality Analyst", "IoT Data Analyst",
    "Sports Data Scientist", "Retail Data Analyst", "Actuarial Data Scientist",
    "Data Science Manager", "AI Product Manager", "Data Analytics Lead",
    "Public Policy Data Analyst", "Transportation Data Analyst", "HR Data Analyst",
    "Climate Data Scientist", "Geospatial Data Analyst", "Open Source Contributor (Data)",
    "Data Science Instructor", "AI Ethicist", "Knowledge Graph Engineer",
    "Data Consultant", "Personalized Systems Specialist"
    ]
    text_lower = text.lower()
    role_keywords = [role.lower() for role in role_keywords]

    for role in role_keywords:
        if role in text_lower:
            return role.title()

    return "Unknown"
def extract_summary(text):
    # Use case-insensitive regex to find 'Summary' and 'Skills'
    pattern = re.compile(r'Summary(.*?)Core Skills', re.DOTALL | re.IGNORECASE)
    match = pattern.search(text)
    if match:
        summary = match.group(1).strip()
        return summary
    return ""
def parse_resume(file_path):
    text = extract_text_from_pdf(file_path)


    return {
        "summary": extract_summary(text),
        "name": extract_name(text),
        "email": extract_email(text),
        "phone": extract_phone(text),
        "links": "; ".join(extract_links(text)),
        "role": extract_role(text),
        "skills": "; ".join(extract_skills(text)),
        "education": extract_degrees(text),
        "experience_years": extract_experience_years(text)
    }

# --- Test on your resume
if __name__ == "__main__":
    parsed = parse_resume("/content/aresumes_folder/cv_1.pdf")  # update if filename changes
    from pprint import pprint
    pprint(parsed)


{'education': 'Bachelor of Science in Digital Transformation',
 'email': 'idpzoxmg@email.com',
 'experience_years': 14,
 'links': 'https://linkedin.com/in/wuowchpyrr; https://github.com/qdhtteejif',
 'name': 'Charlie Green',
 'phone': '+1 9841380772\n',
 'role': 'Statistical Programmer',
 'skills': 'sql; tableau; power bi; r; postgresql; c++; machine learning; '
           'statistics; hadoop; git; docker; model evaluation; model '
           'deployment; hugging face',
 'summary': 'A dedicated and results-driven Statistical Programmer with over '
            '14 years of experience in data\n'
            'analysis, machine learning, and predictive modeling.\n'
            'Skilled in transforming business needs into\n'
            'technical solutions using modern data science tools and '
            'practices. Passionate about solving real-\n'
            'world problems through data-driven approaches and delivering '
            'measurable outcomes.'}


In [16]:
#read 1000 files

import os
import csv
from tqdm import tqdm

def write_to_csv(data, csv_filename):
    fieldnames = ["name", "email", "phone", "links", "role", "skills", "education", "experience_years", "summary"]
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for entry in data:
            writer.writerow(entry)

In [18]:
def process_resumes_in_folder(folder_path, output_csv="parsed_resumes.csv"):
    parsed_data = []
    pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]

    for pdf_file in tqdm(pdf_files[:1000], desc="Parsing resumes"):
        file_path = os.path.join(folder_path, pdf_file)
        try:
            data = parse_resume(file_path)
            parsed_data.append(data)
        except Exception as e:
            print(f"Error processing {pdf_file}: {e}")

    write_to_csv(parsed_data, output_csv)
    print(f"\nDone. Parsed {len(parsed_data)} files and saved to {output_csv}")

In [19]:
if __name__ == "__main__":
    folder_path = "/content/aresumes_folder"  # change to your folder containing PDFs
    process_resumes_in_folder(folder_path)

Parsing resumes: 100%|██████████| 1000/1000 [00:12<00:00, 81.26it/s]


Done. Parsed 1000 files and saved to parsed_resumes.csv
